In [ ]:
# Import thư viện
import json
import concurrent.futures
import re
from textwrap import dedent
from statistics import mean
from dotenv import load_dotenv
from anthropic import Anthropic

In [ ]:
# Khởi tạo Client và các hàm hỗ trợ

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"


def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [ ]:
# Công cụ tạo báo cáo tiếng Việt
def generate_prompt_evaluation_report(evaluation_results):
    total_tests = len(evaluation_results)
    scores = [result["score"] for result in evaluation_results]
    avg_score = mean(scores) if scores else 0
    max_possible_score = 10
    pass_rate = (
        100 * len([s for s in scores if s >= 7]) / total_tests
        if total_tests
        else 0
    )

    html = f"""
    <!DOCTYPE html>
    <html lang="vi">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Báo Cáo Đánh Giá Prompt</title>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
            }}
            .header {{
                background-color: #f0f0f0;
                padding: 20px;
                border-radius: 5px;
                margin-bottom: 20px;
            }}
            .summary-stats {{
                display: flex;
                justify-content: space-between;
                flex-wrap: wrap;
                gap: 10px;
            }}
            .stat-box {{
                background-color: #fff;
                border-radius: 5px;
                padding: 15px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                flex-basis: 30%;
                min-width: 200px;
            }}
            .stat-value {{
                font-size: 24px;
                font-weight: bold;
                margin-top: 5px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            th {{
                background-color: #4a4a4a;
                color: white;
                text-align: left;
                padding: 12px;
            }}
            td {{
                padding: 10px;
                border-bottom: 1px solid #ddd;
                vertical-align: top;
            }}
            tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            .output-cell {{
                white-space: pre-wrap;
            }}
            .score {{
                font-weight: bold;
                padding: 5px 10px;
                border-radius: 3px;
                display: inline-block;
            }}
            .score-high {{
                background-color: #c8e6c9;
                color: #2e7d32;
            }}
            .score-medium {{
                background-color: #fff9c4;
                color: #f57f17;
            }}
            .score-low {{
                background-color: #ffcdd2;
                color: #c62828;
            }}
            .output {{
                overflow: auto;
                white-space: pre-wrap;
            }}
            .output pre {{
                background-color: #f5f5f5;
                border: 1px solid #ddd;
                border-radius: 4px;
                padding: 10px;
                margin: 0;
                font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
                font-size: 14px;
                line-height: 1.4;
                color: #333;
                box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
                overflow-x: auto;
                white-space: pre-wrap;
                word-wrap: break-word;
            }}
            td {{
                width: 20%;
            }}
            .score-col {{
                width: 80px;
            }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>Báo Cáo Đánh Giá Prompt</h1>
            <div class="summary-stats">
                <div class="stat-box">
                    <div>Tổng Số Test Case</div>
                    <div class="stat-value">{total_tests}</div>
                </div>
                <div class="stat-box">
                    <div>Điểm Trung Bình</div>
                    <div class="stat-value">{avg_score:.1f} / {max_possible_score}</div>
                </div>
                <div class="stat-box">
                    <div>Tỷ Lệ Đạt (≥7)</div>
                    <div class="stat-value">{pass_rate:.1f}%</div>
                </div>
            </div>
        </div>
        <table>
            <thead>
                <tr>
                    <th>Kịch Bản</th>
                    <th>Đầu Vào Prompt</th>
                    <th>Tiêu Chí Giải Pháp</th>
                    <th>Kết Quả</th>
                    <th>Điểm</th>
                    <th>Lý Luận</th>
                </tr>
            </thead>
            <tbody>
    """

    for result in evaluation_results:
        prompt_inputs_html = "<br>".join(
            [
                f"<strong>{key}:</strong> {value}"
                for key, value in result["test_case"]["prompt_inputs"].items()
            ]
        )

        criteria_string = "<br>• ".join(
            result["test_case"]["solution_criteria"]
        )

        score = result["score"]
        if score >= 8:
            score_class = "score-high"
        elif score <= 5:
            score_class = "score-low"
        else:
            score_class = "score-medium"

        html += f"""
            <tr>
                <td>{result["test_case"]["scenario"]}</td>
                <td class="prompt-inputs">{prompt_inputs_html}</td>
                <td class="criteria">• {criteria_string}</td>
                <td class="output"><pre>{result["output"]}</pre></td>
                <td class="score-col"><span class="score {score_class}">{score}</span></td>
                <td class="reasoning">{result["reasoning"]}</td>
            </tr>
        """

    html += """
            </tbody>
        </table>
    </body>
    </html>
    """

    return html

In [ ]:
# Triển khai PromptEvaluator với prompt tiếng Việt
class PromptEvaluator:
    def __init__(self, max_concurrent_tasks=3):
        self.max_concurrent_tasks = max_concurrent_tasks

    def render(self, template_string, variables):
        placeholders = re.findall(r"{([^{}]+)}", template_string)
        result = template_string
        for placeholder in placeholders:
            if placeholder in variables:
                result = result.replace(
                    "{" + placeholder + "}", str(variables[placeholder])
                )
        return result.replace("{{", "{").replace("}}", "}")

    def generate_unique_ideas(self, task_description, prompt_inputs_spec, num_cases):
        """Tạo danh sách ý tưởng độc đáo cho test case dựa trên mô tả tác vụ"""
        prompt = """
        Tạo {num_cases} ý tưởng độc đáo và đa dạng để test một prompt thực hiện tác vụ sau:
        
        <mô_tả_tác_vụ>
        {task_description}
        </mô_tả_tác_vụ>

        Prompt sẽ nhận các đầu vào sau:
        <đầu_vào_prompt>
        {prompt_inputs_spec}
        </đầu_vào_prompt>
        
        Mỗi ý tưởng nên đại diện cho một kịch bản hoặc ví dụ riêng biệt để test các khía cạnh khác nhau của tác vụ.
        
        Định dạng đầu ra:
        Cung cấp phản hồi dưới dạng mảng JSON có cấu trúc, trong đó mỗi item là mô tả ngắn gọn về ý tưởng.
        
        Đảm bảo mỗi ý tưởng:
        - Rõ ràng khác biệt với các ý tưởng khác
        - Liên quan đến mô tả tác vụ
        - Đủ cụ thể để hướng dẫn tạo test case đầy đủ
        - Nhanh chóng giải quyết mà không cần tính toán phức tạp hoặc xử lý nhiều bước
        - Có thể giải quyết với không quá 400 token đầu ra

        Nhớ rằng, chỉ tạo {num_cases} ý tưởng độc đáo
        """

        system_prompt = "Bạn là một nhà thiết kế kịch bản test chuyên tạo các kịch bản test đa dạng và độc đáo."

        example_prompt_inputs = ""
        for key, value in prompt_inputs_spec.items():
            val = value.replace("\n", "\\n")
            example_prompt_inputs += f'"{key}": str # {val},'

        rendered_prompt = self.render(
            dedent(prompt),
            {
                "task_description": task_description,
                "num_cases": num_cases,
                "prompt_inputs_spec": example_prompt_inputs,
            },
        )

        messages = []
        add_user_message(messages, rendered_prompt)
        add_assistant_message(messages, "```json")
        text = chat(
            messages,
            stop_sequences=["```"],
            system=system_prompt,
            temperature=1.0,
        )

        return json.loads(text)

    def generate_test_case(self, task_description, idea, prompt_inputs_spec={}):
        """Tạo một test case dựa trên mô tả tác vụ và ý tưởng cụ thể"""

        example_prompt_inputs = ""
        for key, value in prompt_inputs_spec.items():
            val = value.replace("\n", "\\n")
            example_prompt_inputs += f'"{key}": "GIÁ_TRỊ_VÍ_DỤ", // {val}\n'

        allowed_keys = ", ".join(
            [f'"{key}"' for key in prompt_inputs_spec.keys()]
        )

        prompt = """
        Tạo một test case chi tiết cho đánh giá prompt dựa trên:
        
        <mô_tả_tác_vụ>
        {task_description}
        </mô_tả_tác_vụ>
        
        <ý_tưởng_cụ_thể>
        {idea}
        </ý_tưởng_cụ_thể>
        
        <khóa_đầu_vào_được_phép>
        {allowed_keys}
        </khóa_đầu_vào_được_phép>
        
        Định dạng đầu ra:
        ```json
        {{
            "prompt_inputs": {{
            {example_prompt_inputs}
            }},
            "solution_criteria": ["tiêu chí 1", "tiêu chí 2", ...]
        }}
        ```
        
        YÊU CẦU QUAN TRỌNG:
        - Bạn CHỈ ĐƯỢC sử dụng các khóa đầu vào chính xác sau trong prompt_inputs: {allowed_keys}        
        - KHÔNG thêm bất kỳ khóa bổ sung nào vào prompt_inputs
        - Tất cả khóa được liệt kê trong khóa_đầu_vào_được_phép phải được bao gồm trong phản hồi
        - Làm cho test case thực tế và hữu ích về mặt thực tiễn
        - Bao gồm các tiêu chí giải pháp có thể đo lường và súc tích
        - Test case nên được điều chỉnh theo ý tưởng cụ thể được cung cấp
        - Nhanh chóng giải quyết mà không cần tính toán phức tạp hoặc xử lý nhiều bước
        - Có thể giải quyết với không quá 400 token đầu ra
        """

        system_prompt = "Bạn là một người tạo test case chuyên thiết kế các kịch bản đánh giá."

        rendered_prompt = self.render(
            dedent(prompt),
            {
                "allowed_keys": allowed_keys,
                "task_description": task_description,
                "idea": idea,
                "example_prompt_inputs": example_prompt_inputs,
            },
        )

        messages = []
        add_user_message(messages, rendered_prompt)
        add_assistant_message(messages, "```json")
        text = chat(
            messages,
            stop_sequences=["```"],
            system=system_prompt,
            temperature=0.7,
        )

        test_case = json.loads(text)
        test_case["task_description"] = task_description
        test_case["scenario"] = idea

        return test_case

    def generate_dataset(self, task_description, prompt_inputs_spec={}, num_cases=1, output_file="dataset.json"):
        """Tạo test dataset dựa trên mô tả tác vụ và lưu vào file"""
        ideas = self.generate_unique_ideas(
            task_description, prompt_inputs_spec, num_cases
        )

        dataset = []
        completed = 0
        total = len(ideas)
        last_reported_percentage = 0

        with concurrent.futures.ThreadPoolExecutor(
            max_workers=self.max_concurrent_tasks
        ) as executor:
            future_to_idea = {
                executor.submit(
                    self.generate_test_case,
                    task_description,
                    idea,
                    prompt_inputs_spec,
                ): idea
                for idea in ideas
            }

            for future in concurrent.futures.as_completed(future_to_idea):
                try:
                    result = future.result()
                    completed += 1
                    current_percentage = int((completed / total) * 100)
                    milestone_percentage = (current_percentage // 20) * 20

                    if milestone_percentage > last_reported_percentage:
                        print(f"Đã tạo {completed}/{total} test case")
                        last_reported_percentage = milestone_percentage

                    dataset.append(result)
                except Exception as e:
                    print(f"Lỗi khi tạo test case: {e}")

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(dataset, f, indent=2, ensure_ascii=False)

        return dataset

    def grade_output(self, test_case, output, extra_criteria):
        """Chấm điểm đầu ra của test case sử dụng model"""

        prompt_inputs = ""
        for key, value in test_case["prompt_inputs"].items():
            val = value.replace("\n", "\\n")
            prompt_inputs += f'"{key}":"{val}",\n'

        extra_criteria_section = ""
        if extra_criteria:
            extra_criteria_template = """
            Yêu cầu bắt buộc - BẤT KỲ VI PHẠM NÀO CŨNG DẪN ĐẾN THẤT BẠI TỰ ĐỘNG (điểm 3 hoặc thấp hơn):
            <tiêu_chí_quan_trọng_bổ_sung>
            {extra_criteria}
            </tiêu_chí_quan_trọng_bổ_sung>
            """
            extra_criteria_section = self.render(
                dedent(extra_criteria_template),
                {"extra_criteria": extra_criteria},
            )

        eval_template = """
        Nhiệm vụ của bạn là đánh giá giải pháp do AI tạo ra sau đây với ĐỘ NGHIÊM NGẶT CỰC CAO.

        Mô tả tác vụ gốc:
        <mô_tả_tác_vụ>
        {task_description}
        </mô_tả_tác_vụ>

        Đầu vào tác vụ gốc:
        <đầu_vào_tác_vụ>
        {{ {prompt_inputs} }}
        </đầu_vào_tác_vụ>

        Giải pháp cần đánh giá:
        <giải_pháp>
        {output}
        </giải_pháp>

        Tiêu chí bạn nên sử dụng để đánh giá giải pháp:
        <tiêu_chí>
        {solution_criteria}
        </tiêu_chí>

        {extra_criteria_section}

        Hướng dẫn chấm điểm:
        * Điểm 1-3: Giải pháp không đáp ứng một hoặc nhiều yêu cầu BẮT BUỘC
        * Điểm 4-6: Giải pháp đáp ứng tất cả yêu cầu bắt buộc nhưng có thiếu sót đáng kể trong tiêu chí phụ
        * Điểm 7-8: Giải pháp đáp ứng tất cả yêu cầu bắt buộc và hầu hết tiêu chí phụ, với các vấn đề nhỏ
        * Điểm 9-10: Giải pháp đáp ứng tất cả tiêu chí bắt buộc và phụ

        Định dạng đầu ra
        Cung cấp đánh giá của bạn dưới dạng đối tượng JSON có cấu trúc với các trường sau:
        - "strengths": Mảng 1-3 điểm mạnh chính
        - "weaknesses": Mảng 1-3 lĩnh vực chính cần cải thiện
        - "reasoning": Giải thích súc tích về đánh giá tổng thể của bạn
        - "score": Số từ 1-10

        Phản hồi bằng JSON. Giữ phản hồi súc tích và trực tiếp.
        """

        eval_prompt = self.render(
            dedent(eval_template),
            {
                "task_description": test_case["task_description"],
                "prompt_inputs": prompt_inputs,
                "output": output,
                "solution_criteria": "\n".join(test_case["solution_criteria"]),
                "extra_criteria_section": extra_criteria_section,
            },
        )

        messages = []
        add_user_message(messages, eval_prompt)
        add_assistant_message(messages, "```json")
        eval_text = chat(
            messages,
            stop_sequences=["```"],
            temperature=0.0,
        )
        return json.loads(eval_text)

    def run_test_case(self, test_case, run_prompt_function, extra_criteria=None):
        """Chạy test case và chấm điểm kết quả"""
        output = run_prompt_function(test_case["prompt_inputs"])

        model_grade = self.grade_output(test_case, output, extra_criteria)
        model_score = model_grade["score"]
        reasoning = model_grade["reasoning"]

        return {
            "output": output,
            "test_case": test_case,
            "score": model_score,
            "reasoning": reasoning,
        }

    def run_evaluation(self, run_prompt_function, dataset_file, extra_criteria=None, json_output_file="output.json", html_output_file="output.html"):
        """Chạy đánh giá trên tất cả test case trong dataset"""
        with open(dataset_file, "r", encoding="utf-8") as f:
            dataset = json.load(f)

        results = []
        completed = 0
        total = len(dataset)
        last_reported_percentage = 0

        with concurrent.futures.ThreadPoolExecutor(
            max_workers=self.max_concurrent_tasks
        ) as executor:
            future_to_test_case = {
                executor.submit(
                    self.run_test_case,
                    test_case,
                    run_prompt_function,
                    extra_criteria,
                ): test_case
                for test_case in dataset
            }

            for future in concurrent.futures.as_completed(future_to_test_case):
                result = future.result()
                completed += 1
                current_percentage = int((completed / total) * 100)
                milestone_percentage = (current_percentage // 20) * 20

                if milestone_percentage > last_reported_percentage:
                    print(f"Đã chấm điểm {completed}/{total} test case")
                    last_reported_percentage = milestone_percentage
                results.append(result)

        average_score = mean([result["score"] for result in results])
        print(f"Điểm trung bình: {average_score}")

        with open(json_output_file, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

        html = generate_prompt_evaluation_report(results)
        with open(html_output_file, "w", encoding="utf-8") as f:
            f.write(html)

        return results

In [ ]:
# Tạo một instance của PromptEvaluator
# Tăng max_concurrent_tasks để có tính đồng thời cao hơn, nhưng cẩn thận với lỗi rate limit!
evaluator = PromptEvaluator(max_concurrent_tasks=1)

In [ ]:
dataset = evaluator.generate_dataset(
    # Mô tả mục đích hoặc mục tiêu của prompt bạn đang cố gắng test
    task_description="Viết kế hoạch bữa ăn 1 ngày gọn gàng, súc tích cho một vận động viên",
    # Mô tả các đầu vào khác nhau mà prompt của bạn yêu cầu
    prompt_inputs_spec={
        "height": "Chiều cao của vận động viên tính bằng cm",
        "weight": "Cân nặng của vận động viên tính bằng kg",
        "goal": "Mục tiêu của vận động viên",
        "restriction": "Hạn chế chế độ ăn của vận động viên"
    },
    # Nơi ghi dataset được tạo
    output_file="dataset.vn.json",
    # Số lượng test case cần tạo (khuyến nghị giữ số này thấp nếu bạn gặp lỗi rate limit)
    num_cases=3,
)

In [ ]:
# Định nghĩa và chạy prompt bạn muốn đánh giá, trả về đầu ra model thô
# Hàm này được thực thi một lần cho mỗi test case
def run_prompt(prompt_inputs):
    prompt = f"""
    Tạo kế hoạch bữa ăn một ngày cho vận động viên đáp ứng các hạn chế chế độ ăn của họ.

    <thông_tin_vận_động_viên>
    - Chiều cao: {prompt_inputs["height"]}
    - Cân nặng: {prompt_inputs["weight"]}
    - Mục tiêu: {prompt_inputs["goal"]}
    - Hạn chế chế độ ăn: {prompt_inputs["restriction"]}
    </thông_tin_vận_động_viên>

    Hướng dẫn:
    1. Bao gồm lượng calo hàng ngày chính xác
    2. Hiển thị lượng protein, chất béo và carb
    3. Chỉ định thời gian ăn mỗi bữa
    4. Chỉ sử dụng thực phẩm phù hợp với hạn chế
    5. Liệt kê tất cả khẩu phần bằng gram
    6. Giữ thân thiện với ngân sách nếu được đề cập

    Đây là ví dụ với đầu vào mẫu và ý tưởng về đầu ra lý tưởng:
    <đầu_vào_mẫu>
    chiều cao: 180
    cân nặng: 85
    mục tiêu: Hiệu suất đỉnh cao trong thi đấu CrossFit
    hạn chế: Không gluten
    </đầu_vào_mẫu>

    <đầu_ra_lý_tưởng>
    Kế Hoạch Bữa Ăn Cho Vận Động Viên CrossFit

    Tính Toán Calo Hàng Ngày:
    - Tỷ Lệ Trao Đổi Chất Cơ Bản (BMR): 1,900 calo
    - Hệ Số Hoạt Động: 1.8 (Tập Luyện Cường Độ Cao)
    - Tổng Calo Hàng Ngày: 3,420 calo

    Phân Chia Chất Dinh Dưỡng Đa Lượng:
    - Protein: 30% (257g)
    - Carbohydrate: 40% (342g)
    - Chất béo: 30% (114g)

    Kế Hoạch Bữa Ăn:

    🍳 Bữa Sáng (7:00 AM) - 700 calo
    - Cháo quinoa (150g)
    - 4 lòng trắng trứng + 2 trứng nguyên quả (200g)
    - Hoa quả tổng hợp (100g)
    - Bơ hạnh nhân (30g)
    - Hạt chia (15g)

    🥗 Bữa Phụ Giữa Sáng (10:00 AM) - 400 calo
    - Sữa chua Hy Lạp (không béo, không gluten) (200g)
    - Hạnh nhân (30g)
    - Chuối (100g)

    🍽️ Bữa Trưa (1:00 PM) - 800 calo
    - Ức gà nướng (200g)
    - Khoai lang (150g)
    - Rau củ nướng tổng hợp (200g)
    - Sốt dầu ô liu (30g)

    🥤 Bữa Phụ Trước Tập (4:00 PM) - 350 calo
    - Protein shake gạo (50g)
    - Yến mạch không gluten (50g)
    - Việt quất (100g)

    🏋️ Bữa Sau Tập (6:00 PM) - 750 calo
    - Cá hồi tự nhiên (180g)
    - Quinoa (100g)
    - Bông cải xanh hấp (150g)
    - Bơ (50g)

    🌙 Bữa Phụ Tối (9:00 PM) - 420 calo
    - Ức gà tây (100g)
    - Phô mai cottage (ít béo, 150g)
    - Hạnh nhân (25g)

    Hydration:
    - Uống nước: 4-5 lít mỗi ngày
    - Khuyên dùng thêm chất điện giải

    Mẹo Tiết Kiệm:
    - Mua protein số lượng lớn
    - Chọn rau theo mùa
    - Sử dụng hoa quả đông lạnh
    - Mua thực phẩm không hỏng với số lượng lớn

    Cân Nhắc Dinh Dưỡng:
    - 100% không gluten
    - Protein cao cho phục hồi cơ bắp
    - Carbohydrate phức tạp cho năng lượng bền vững
    - Chất béo lành mạnh cho cân bằng hormone

    Khuyến Nghị Thực Phẩm Bổ Sung:
    - Multivitamin
    - Omega-3
    - Vitamin D
    - Magie

    Ghi Chú:
    - Điều chỉnh khẩu phần theo cường độ tập luyện hàng ngày
    - Tham khảo ý kiến chuyên gia dinh dưỡng cho kế hoạch cá nhân hóa
    - Theo dõi thành phần cơ thể và hiệu suất
    </đầu_ra_lý_tưởng>

    Giải pháp đáp ứng tất cả yêu cầu bắt buộc với kế hoạch bữa ăn chính xác, tập trung vào vận động viên. Nó cung cấp chiến lược dinh dưỡng hàng ngày hoàn chỉnh với lượng calo chính xác, phân chia chất dinh dưỡng đa lượng, thời gian bữa ăn, và cân nhắc không gluten. Kế hoạch được thiết kế riêng để hỗ trợ hiệu suất CrossFit cường độ cao với dinh dưỡng cân bằng và thành phần bữa ăn chiến lược.
    """

    messages = []
    add_user_message(messages, prompt)
    return chat(messages)

In [ ]:
results = evaluator.run_evaluation(
    run_prompt_function=run_prompt, 
    dataset_file="dataset.vn.json",
    extra_criteria="""
    Đầu ra nên bao gồm:
    - Tổng lượng calo hàng ngày
    - Phân chia chất dinh dưỡng đa lượng (macro)
    - Bữa ăn với thực phẩm chính xác, khẩu phần và thời gian
    """,
    json_output_file="output.vn.json",
    html_output_file="output.vn.html"
)